<a href="https://colab.research.google.com/github/Sourenm/GenAI_SalesManager/blob/main/Solve_Business_Problems_with_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preamble: Importing required libraries, housekeeping, admin stuff

In [ ]:
## PREAMBLE:
# 1. import all the libraries,
# 2. create the OpenAI client,
# 3. read the material from google spreadsheet and load it into dataframes,
# 4. create vector indices for product names and their descriptions to be later used
# 5. creating a new spreadsheet with the required sheets

from openai import OpenAI
import pandas as pd
from IPython.display import display
import ast
import numpy as np
import json
import re
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

output_document = gc.create('Solving Business Problems with AI - Output')

# Create 'email-classification' sheet
email_classification_sheet = output_document.add_worksheet(title="email-classification", rows=50, cols=2)
email_classification_sheet.update([['email ID', 'category']], 'A1:B1')

# Create 'order-status' sheet
order_status_sheet = output_document.add_worksheet(title="order-status", rows=50, cols=4)
order_status_sheet.update([['email ID', 'product ID', 'quantity', 'status']], 'A1:D1')

# Create 'order-response' sheet
order_response_sheet = output_document.add_worksheet(title="order-response", rows=50, cols=2)
order_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Create 'inquiry-response' sheet
inquiry_response_sheet = output_document.add_worksheet(title="inquiry-response", rows=50, cols=2)
inquiry_response_sheet.update([['email ID', 'response']], 'A1:B1')

# delete Sheet1
sheet_to_delete = output_document.worksheet("Sheet1")
output_document.del_worksheet(sheet_to_delete)


# Share the spreadsheet publicly
output_document.share('', perm_type='anyone', role='reader')

# This is the solution output link, paste it into the submission form
print(f"Shareable link: https://docs.google.com/spreadsheets/d/{output_document.id}")

client = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url= # ENTER URL HERE (OPTIONAL)
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key= # ENTER KEY HERE
)

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

# currently pointing to a copy of the Google sheet on my Google drive which is accessible for read via link, change if necessary
document_id = # Insert Document ID from Google Sheets here: The Google Sheet must have two sheets: prducts, emails. full descriptin is in the README.md
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

def compute_product_name_and_description_embeddings(products_df):
    name_embeddings = []
    description_embeddings = []

    for _, row in products_df.iterrows():
        name_text = row['name']
        desc_text = row['description']

        name_resp = client.embeddings.create(
            model="text-embedding-ada-002",
            input=name_text
        )
        desc_resp = client.embeddings.create(
            model="text-embedding-ada-002",
            input=desc_text
        )

        name_embeddings.append(name_resp.data[0].embedding)
        description_embeddings.append(desc_resp.data[0].embedding)

    products_df["name_vector"] = name_embeddings
    products_df["description_vector"] = description_embeddings
    return products_df

products_df = compute_product_name_and_description_embeddings(products_df)

Shareable link: https://docs.google.com/spreadsheets/d/1U94dw4HmYFza10s4LkxSwMTXPioCKwozCvWxXTnB40k


# Task 1. Classify emails

In [ ]:
def classify_email(subject, body):
    prompt = f"Classify the following email as either 'product inquiry' or 'order request'.\n\nSubject: {subject}\nBody: {body}\n\n only say 'product inquiry' if it is a product inquiry or 'order request' if it is an order request and say nothing else."

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    # Extract and return the classification
    final_response = 'order request' if 'order request' in response.choices[0].message.content.lower() else 'product inquiry'
    return final_response

# Apply classification to each email in the DataFrame
emails_df['category'] = emails_df.apply(
    lambda row: classify_email(row['subject'], row['message']), axis=1
)

# populate the appropriate sheet
set_with_dataframe(email_classification_sheet, emails_df[["email_id", "category"]])

# Task 2. Process order requests

In [ ]:
def _ext_ord(email_body):
    prompt = f"Extract all product_id and quantity pairs from the following order request email body. Format the response as a list of dictionaries: '[{{'product_id': <id>, 'quantity': <quantity>}}]'.\n\nYour only response MUST be the list of dictionaries ONLY. No additional text required. If the amount requested refers to all of the remaining stock assign -1 to quantity. If the amount requested is a range for a certain product use the minimum number and do not create multiple entries for the same order. If the amount requested are things like 'a pair' or 'a dozen' or etc. set the quantity accordingly and remember 'a pair' for shoes means 1. If no product ID can be found, retrieve a product name as the product ID instead. Finally, if the customer is not ordering some of the items and just talking about them or talking about wanting to buy them in the future and not now do not include them in the returned list of items and only return items that the customer is ordering now.\n\nEmail Body: {email_body}"

    # Send the request to the OpenAI model to extract multiple orders
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

def extract_orders(email_body):
    count = 0
    flag = False
    while count < 10:
        # Extract the multiple orders from the response
        extracted_info = _ext_ord(email_body)

        # Safely evaluate the extracted_info to a list
        try:
            orders = ast.literal_eval(extracted_info)  # Convert string to a list of dicts
        except (ValueError, SyntaxError):
            orders = []  # If parsing fails, treat it as empty

        if len(orders) > 0:  # Now check if the orders list is non-empty
            flag = True
            break

        count += 1

    return orders

# Function to process each order request using OpenAI's extraction
def process_multiple_orders_with_openai(email_id, orders):
    order_lines = []

    # Loop through each order in the list of extracted orders
    for order in orders:
        product_id_or_name = order['product_id']
        quantity = int(order['quantity'])

        # Try to find the product by product_id first
        product = products_df[products_df['product_id'] == product_id_or_name.upper()]
        if product.empty:
            product = products_df[products_df['product_id'] == product_id_or_name.upper().replace(" ", "")]

        # If product is not found by product_id, treat the product_id_or_name as a name and use fuzzy matching
        if product.empty:
            # Embed the product name as a query vector
            query_embedding = client.embeddings.create(
                model="text-embedding-ada-002",
                input=product_id_or_name
            ).data[0].embedding

            # Compute cosine similarity between query and each name_vector
            def cosine_similarity(a, b):
                a = np.array(a)
                b = np.array(b)
                return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

            similarities = products_df['name_vector'].apply(lambda vec: cosine_similarity(vec, query_embedding))
            best_match_idx = similarities.idxmax()
            best_score = similarities[best_match_idx]

            if best_score >= 0.85:  # You can tune this threshold
                product = products_df.loc[[best_match_idx]]
            else:
                print(f"No good vector match found for '{product_id_or_name}' (score: {best_score:.2f}). Skipping order and changing its category to 'product inquiry'.")
                continue

        # After matching by ID or name, get the stock and product details
        stock_available = product['stock'].values[0]

        # Handle the order quantity
        if quantity == -1:  # Special case where quantity is -1 (to get all available stock)
            if stock_available > 0:
                quantity = stock_available

        # Check if the stock is sufficient for the order
        if stock_available >= quantity:
            status = "created"
            new_stock = stock_available - quantity
        else:
            status = "out of stock"
            new_stock = stock_available  # No stock change in case of out of stock

        # Update the stock in the products DataFrame
        product_id = product['product_id'].values[0]
        products_df.loc[products_df['product_id'] == product_id, 'stock'] = new_stock

        # Append the order line to the order lines list
        order_lines.append({
            'email_id': email_id,
            'product_id': product_id,
            'quantity': quantity,
            'status': status
        })

    return order_lines

# Function to process all order requests from the emails
def process_all_orders():
    order_status_data = []  # List to hold order status details

    # Loop through all emails and process order requests
    for _, email in emails_df[emails_df['category'] == 'order request'].iterrows():
        email_id = email['email_id']
        email_body = email['message']

        # Extract multiple order details (product_id and quantity) using OpenAI
        orders = extract_orders(email_body)

        if orders:
            # Process each extracted order
            order_lines = process_multiple_orders_with_openai(email_id, orders)
            order_status_data.extend(order_lines)
        else:
            emails_df.loc[emails_df['email_id'] == email_id, 'category'] = 'product inquiry'

    # Convert the collected order status data to a DataFrame
    order_status_df = pd.DataFrame(order_status_data, columns=['email_id', 'product_id', 'quantity', 'status'])

    return order_status_df

# Process the orders and update the order status DataFrame
order_status_df = process_all_orders()

# populate the appropriate sheet
set_with_dataframe(order_status_sheet, order_status_df)

# Task 3. Handle product inquiry

## 3.1 Respond to Order Request

In [ ]:
def generate_response_for_order(email_id, order_lines):
    # Split orders into fulfilled and out of stock
    fulfilled_orders = [line for line in order_lines if line['status'] == 'created']
    out_of_stock_orders = [line for line in order_lines if line['status'] == 'out of stock']

    # Build the order details for the OpenAI prompt
    fulfilled_details = "\n".join([f"- Product ID {order['product_id']}: {order['quantity']} unit(s)" for order in fulfilled_orders])
    out_of_stock_details = "\n".join([f"- Product ID {order['product_id']}: {order['quantity']} unit(s)" for order in out_of_stock_orders])

    # Prepare the prompt to send to OpenAI for generating the response
    if fulfilled_orders and not out_of_stock_orders:
        prompt = f"Create a professional email response informing the customer that their order is fully fulfilled. The following items have been shipped:\n{fulfilled_details}\n\nThe tone should be friendly and professional."
    else:
        prompt = f"Create a professional email response informing the customer that the following items are out of stock:\n{out_of_stock_details}\n\nThe following items have been shipped:\n{fulfilled_details}\n\nApologize for the inconvenience and offer alternatives (e.g., waiting for restock or finding alternatives). The tone should be friendly and professional."

    # Request OpenAI to generate the response
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    # Extract and return the response content
    return response.choices[0].message.content.strip()

# Function to generate responses for all emails
def generate_all_responses():
    order_responses = []  # List to hold responses

    # Loop through all emails and generate responses for order requests
    for _, email in emails_df[emails_df['category'] == 'order request'].iterrows():
        email_id = email['email_id']

        # Get the corresponding order lines from the order_status_df
        order_lines = order_status_df[order_status_df['email_id'] == email_id].to_dict('records')

        # Generate the response for the email using OpenAI
        response = generate_response_for_order(email_id, order_lines)

        # Append the response to the order_responses list
        order_responses.append({
            'email_id': email_id,
            'response': response
        })

    # Convert the responses to a DataFrame
    order_response_df = pd.DataFrame(order_responses, columns=['email_id', 'response'])

    return order_response_df

# Generate all responses and display the order-response DataFrame
order_response_df = generate_all_responses()

# populate the appropriate sheet
set_with_dataframe(order_response_sheet, order_response_df)

## 3.2 Respond to product inquiries

In [ ]:
def classify_inquiry_category(email_body):
    CATEGORY_MAP = {
        "accessories": "Accessories",
        "mens accessories": "Accessories",
        "bags": "Bags",
        "kids clothing": "Kids' Clothing",
        "mens clothing": "Men's Clothing",
        "mens shoes": "Men's Shoes",
        "womens clothing": "Women's Clothing",
        "womens shoes": "Women's Shoes",
        "unknown": "Unknown"
    }

    prompt = (
        "Classify the following inquiry into one or more of these categories: "
        "accessories, bags, kids clothing, mens clothing, mens shoes, womens clothing, womens shoes, or unknown. "
        "Return a list containing only the category names mentioned in the email. Return as normal words seperated by commas.\n\n"
        f"Email Body: {email_body}\n\nCategories:"
    )

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    raw_text = response.choices[0].message.content.strip()

    # Fallback parser — split and normalize manually
    raw_text = raw_text.lower()
    extracted = [x.strip() for x in re.split(r"[,\n]+", raw_text)]

    # Map to correct categories
    final_categories = [CATEGORY_MAP[cat] for cat in extracted]
    final_categories = [cat for cat in final_categories if cat is not None]

    return final_categories if final_categories else ["Unknown"]

def handle_product_inquiry(email_body, categories):
    full_response = ""

    for category in categories:
        normalized_category = category.strip().lower()
        response = f"**Category: {category}**\n"

        if normalized_category == "unknown":
            response += "We couldn't confidently match your inquiry to a product category. Could you please provide more details?\n\n"
        else:
            # Normalize category matching
            def normalize(text):
                return str(text).strip().lower()
            relevant_products = products_df[products_df['category'].apply(normalize) == normalized_category]
            relevant_products_in_stock = relevant_products[relevant_products['stock'] > 0]

            if relevant_products_in_stock.empty:
                response += f"Unfortunately, we currently do not have any items in the '{category}' category available in stock.\n\n"
            else:
                # Embed the email body for vector search
                query_embedding = client.embeddings.create(
                    model="text-embedding-ada-002",
                    input=email_body
                ).data[0].embedding

                # Cosine similarity for ranking
                def cosine_similarity(a, b):
                    a = np.array(a)
                    b = np.array(b)
                    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
                relevant_products_in_stock = relevant_products_in_stock.copy()  # Avoid SettingWithCopyWarning
                relevant_products_in_stock["similarity"] = relevant_products_in_stock["description_vector"].apply(
                    lambda vec: cosine_similarity(vec, query_embedding)
                )

                top_product = relevant_products_in_stock.sort_values(by="similarity", ascending=False).iloc[0]
                product_info = (
                    f"Name: {top_product['name']}\n"
                    f"Description: {top_product['description']}\n"
                    f"Price: ${top_product['price']}\n"
                    f"Stock Available: {top_product['stock']}"
                )

                product_prompt = (
                    f"The customer asked: \"{email_body}\"\n\n"
                    f"Based on this, respond to them about the most relevant product in the {category} category:\n\n{product_info}"
                )

                category_response = client.chat.completions.create(
                    model="gpt-4",
                    messages=[{"role": "user", "content": product_prompt}]
                ).choices[0].message.content

                response += category_response.strip() + "\n\n"

        full_response += response

    return full_response.strip()


def process_inquiries():
    inquiry_responses = []

    for _, row in emails_df[emails_df['category'] == 'product inquiry'].iterrows():
        email_id = row['email_id']
        email_body = row['message']

        # Classify the inquiry category
        category = classify_inquiry_category(email_body)

        # Handle the inquiry based on the category
        response = handle_product_inquiry(email_body, category)

        # Append the result to the inquiry responses list
        inquiry_responses.append({
            'email_id': email_id,
            'response': response
        })

    # Create the DataFrame with responses
    inquiry_response_df = pd.DataFrame(inquiry_responses)
    return inquiry_response_df

inquiry_response_df = process_inquiries()

# populate the appropriate sheet
set_with_dataframe(inquiry_response_sheet, inquiry_response_df)